Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [3]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136912
aml-quickstarts-136912
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [5]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'website_classification'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
website_classification,quick-starts-ws-136912,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "autoML-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded..............................................................................

KeyboardInterrupt: 

## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Website Classification Dataset"
description_text = "Website Classification DataSet for Capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Panth-Shah/nd00333-capstone/master/Dataset/malicious_website_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [8]:
df = dataset.to_pandas_dataframe()
df.describe()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
count,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1.781000e+03,1781.000000,1781.000000,1.781000e+03,1.781000e+03,1781.000000,1780.000000,1781.000000
mean,56.961258,11.111735,16.261089,5.472768,3.060640,2.982339e+03,18.540146,18.746210,1.589255e+04,3.155599e+03,18.540146,2.263483,0.121280
std,27.555586,4.549896,40.500975,21.807327,3.386975,5.605057e+04,41.627173,46.397969,6.986193e+04,5.605378e+04,41.627173,2.930853,0.326544
min,16.000000,5.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,39.000000,8.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
50%,49.000000,10.000000,7.000000,0.000000,2.000000,6.720000e+02,8.000000,9.000000,5.790000e+02,7.350000e+02,8.000000,0.000000,0.000000
75%,68.000000,13.000000,22.000000,5.000000,5.000000,2.328000e+03,26.000000,25.000000,9.806000e+03,2.701000e+03,26.000000,4.000000,0.000000
max,249.000000,43.000000,1194.000000,708.000000,17.000000,2.362906e+06,1198.000000,1284.000000,2.060012e+06,2.362906e+06,1198.000000,20.000000,1.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [9]:
dataset.take(5).to_pandas_dataframe()

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263,None,None,10/10/2015 18:21,None,...,0,2,700,9,10,1153,832,9,2,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087,None,None,None,None,...,7,4,1230,17,19,1265,1230,17,0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,B0_113,17,6,ISO-8859-1,nginx,162,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8,0
4,B0_403,17,6,UTF-8,None,124140,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4,0


In [10]:
from train import preprocess_inputs
X, y = preprocess_inputs(df)

In [13]:
X['result'] = y
X.head()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CONTENT_LENGTH,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,...,WS_west midlands,WS_wi,WS_widestep@mail.ru,WS_wisconsin,WS_worcs,WS_wv,WS_zh,WS_zhejiang,WS_zug,result
0,16,7,263,7,0,2,700,9,10,1153,...,0,0,0,0,0,0,0,0,0,1
1,16,6,15087,17,7,4,1230,17,19,1265,...,0,0,0,0,0,0,0,0,0,0
2,16,6,324,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,6,162,31,22,3,3812,39,37,18784,...,0,0,0,0,0,0,0,0,0,0
4,17,6,124140,57,2,5,4278,61,62,129889,...,0,0,0,0,0,0,0,0,0,0


In [15]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(X).to_csv("data/train_data.csv", index=False)

In [16]:
from azureml.core import Workspace, Dataset
from azureml.data.datapath import DataPath

datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

datastore_path =[
    DataPath(datastore, 'data/train_data.csv')
]

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = Dataset.Tabular.from_delimited_files(path=datastore_path)
label = "result"

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [17]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

project_folder = './capstone-project'

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_data,
                             label_column_name=label,   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             n_cross_validations=5,
                             **automl_settings
                            )

In [ ]:
# Submit your automl run

automl_exp = Experiment(workspace=ws, name="capestone_AutoML")  
automl_run = automl_exp.submit(automl_config, show_output = True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on autoML-compute with default configuration
Running on remote compute: autoML-compute
Parent Run ID: AutoML_da113e46-b788-43ee-a689-dcb7705ed4cf



In [ ]:
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_autoML_run, best_autoML_fitted_model = automl_run.get_output()
print(best_autoML_run)
print(best_autoML_fitted_model)

In [ ]:
get_best_autoML_metrics = best_autoML_run.get_metrics() 
print(get_best_autoML_metrics)

In [ ]:
best_autoML_run.get_file_names()

In [ ]:
# Get your best run and save the model from that run.

best_autoML_run.register_model(model_name = 'best_autoML_model', model_path =  'outputs/model.pkl')

In [ ]:
best_autoML_run.download_file("outputs/model.pkl","outputs/best_model_autoML.pkl")

In [ ]:
best_autoML_run

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [8d934bc8][9564fa22-904b-4d51-a869-d16b92ac3ee2], (This step will run and generate new outputs)
Submitted PipelineRun d0620ce6-fa31-4033-ac9e-a0ab708cc54c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/d0620ce6-fa31-4033-ac9e-a0ab708cc54c?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-135297/workspaces/quick-starts-ws-135297


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: d0620ce6-fa31-4033-ac9e-a0ab708cc54c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/d0620ce6-fa31-4033-ac9e-a0ab708cc54c?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-135297/workspaces/quick-starts-ws-135297
PipelineRun Status: Running


StepRunId: 8e092fb7-5f7d-4c10-878e-f3b71d69e8ee
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml-experiment-1/runs/8e092fb7-5f7d-4c10-878e-f3b71d69e8ee?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-135297/workspaces/quick-starts-ws-135297
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '8e092fb7-5f7d-4c10-878e-f3b71d69e8ee', 'target': 'autoML-compute', 'status': 'Completed', 'startTimeUtc': '2021-01-19T06:19:16.285519Z', 'endTimeUtc': '2021-01-19T06:56:31.125661Z', 'properties': {'azureml.runsource'

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/8e092fb7-5f7d-4c10-878e-f3b71d69e8ee/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_43,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_40,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_30,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_45,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_60,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_0,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_33,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_2,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_41,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_59,...,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_36,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_37,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_39,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_56,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_4,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_12,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_42,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_18,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_31,8e092fb7-5f7d-4c10-878e-f3b71d69e8ee_54
recall_score_weighted,[0.9132018209408195],[0.9083459787556905],[0.8880121396054628],[0.910773899848255],[0.8880121396054628],[0.9116843702579667],[0.9119878603945372],[0.891350531107739],[0.8880121396054628],[0.9159332321699545],...,[0.9125948406676783],[0.9110773899848255],[0.9159332321699545],[0.9110773899848255],[0.8880121396054628],[0.74597875569044],[0.9040971168437025],[0.7089529590288316],[0.9128983308042489],[0.910773899848255]
precision_score_micro,[0.9132018209408195],[0.9083459787556905],[0.8880121396054628],[0.910773899848255],[0.8880121396054628],[0.9116843702579667],[0.9119878603945372],[0.891350531107739],[0.8880121396054628],[0.9159332321699545],...,[0.9125948406676783],[0.9110773899848255],[0.9159332321699545],[0.9110773899848255],[0.8880121396054628],[0.74597875569044],[0.9040971168437025],[0.7089529590288316],[0.9128983308042489],[0.910773899848255]
f1_score_micro,[0.9132018209408195],[0.9083459787556905],[0.8880121396054628],[0.9107738998482551],[0.8880121396054628],[0.9116843702579667],[0.9119878603945372],[0.891350531107739],[0.8880121396054628],[0.9159332321699545],...,[0.9125948406676783],[0.9110773899848255],[0.9159332321699545],[0.9110773899848255],[0.8880121396054628],[0.74597875569044],[0.9040971168437025],[0.7089529590288317],[0.9128983308042489],[0.9107738998482551]
average_precision_score_weighted,[0.9541411484948241],[0.9499599601915379],[0.948588659974036],[0.9501487309224028],[0.9556872458271025],[0.9531771295804466],[0.953625132213001],[0.9302839274463027],[0.9434384837698533],[0.9554343247872955],...,[0.9512709076945858],[0.952268137874364],[0.9523358020200623],[0.9503543255163533],[0.9201069392678627],[0.9238035248097693],[0.9481378382396821],[0.9201307201002716],[0.9526538456636413],[0.9527388952648794]
average_precision_score_micro,[0.9805583578526404],[0.9778699660657124],[0.9755997293287618],[0.978824384448086],[0.9781805163955288],[0.9806603102489483],[0.9804926577142863],[0.9651350920340517],[0.9732705157966541],[0.9813847692561776],...,[0.9797387760522972],[0.9800805516651045],[0.9805253710646454],[0.9780965027036886],[0.9645579445500554],[0.8268089210322341],[0.9767246022580298],[0.835636085228748],[0.9805134721211332],[0.9798758954137006]
recall_score_micro,[0.9132018209408195],[0.9083459787556905],[0.8880121396054628],[0.910773899848255],[0.8880121396054628],[0.9116843702579667],[0.9119878603945372],[0.891350531107739],[0.8880121396054628],[0.9159332321699545],...,[0.9125948406676783],[0.9110773899848255],[0.9159332321699545],[0.9110773899848255],[0.8880121396054628],[0.74597875569044],[0.9040971168437025],[0.7089529590288316],[0.9128983308042489],[0.910773899848255]
log_loss,[0.18012736943904312],[0.3339806816963599],[0.29313359336803707],[0.21321407629825834],[0.34087207391407254],[0.17775706110025447],[0.17977421208783076],[0.2529079217007151],[0.24275331984516882],[0.20389804103104314],...,[0.1879296926979376],[0.18183445704003637],[0.18011527626106247],[0.3210386383780024],[0.2744992358523854],[0.5938207513235457],[0.2184722094165244],[0.5805682106981083],[0.18194487595378997],[0.20906686358242352]
weighted_accuracy,[0.954939715235299],[0.9754280386730326],[0.9843450583187134],[0.95166043

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/8e092fb7-5f7d-4c10-878e-f3b71d69e8ee/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=None,
                                                                  enable_feature_sweeping=None,
                                                                  feature_sweeping_config=None,
                                                                  feature_sweeping_timeout=None,
                                                                  featurization_config=None,
                                                                  force_text_dnn=None,
                                                                  is_cross_validation=None,
                                                                  is_onnx_compatible=None,
                                                                  logger=None,
                                                              

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=None,
                                estimators=[('39',
                                             Pipeline(memory=None,
                                                      steps=[('standardscalerwrapper',
                                                              <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7efcbaeea978>),
                                                             ('xgboostclassifier',
                                                              XGBoostClassifier(ba

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [19]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [20]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [21]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28882,376
1,934,2758


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,af201a79-dd49-4173-a2ee-192d8b009915,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [23]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [25]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  31d430de-b326-494d-ab54-62d3c29045ce


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [26]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …